<a href="https://colab.research.google.com/github/zhrsaghaie/OvarianCancer_Project/blob/master/PCA_on_GSE106817.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

path = "/content/drive/MyDrive/Colab Notebooks/GSE106817_Data.csv"
all_samples = pd.read_csv(path)

y = all_samples["HasCancer"].tolist()
x = all_samples.iloc[:, :-1]

# Load and split data

# X, y = make_moons(noise=0.3, random_state=0)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)


In [ ]:
x_train["MIMAT0027629"][0]


4.109

In [ ]:
y_train[0]


0

In [ ]:
x_train.shape

(691, 2565)

In [ ]:
#Normalization

mm = all_samples.min().min()
all_samples = all_samples + (-mm)
all_samples /= 23.385

In [ ]:
# ---------------------------
# Convert to Gene names
# ----------------------------

import time
import requests
from bs4 import BeautifulSoup

def get_gene_symbol(gene_id):
    # Use the NCBI Gene ID to retrieve information about the gene
    response = requests.get(f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=gene&id={gene_id}&retmode=xml')
    time.sleep(1)
    soup = BeautifulSoup(response.text, 'xml')
    gene_symbol = soup.find('Gene-ref').find('Gene-ref_locus').text

    return gene_symbol



def print_genes(genes):
    for gene_id in genes:
        # Example usage:
        #gene_id = 'MIMAT0019221'
        gene_symbol = get_gene_symbol(gene_id)
        print(f'Gene ID {gene_id} is {gene_symbol}')

In [ ]:
# -----------------
# Using PCA
# -------------------

from sklearn.decomposition import PCA

df = all_samples

pca = PCA(n_components=10)
# Split the data into features (X)
X = df.iloc[:, :-1] # assuming the last column is the target column

# Reduce the number of features to 30 using PCA
pca = PCA(n_components=30)
X_new = pca.fit_transform(X)




print("variance ratio", pca.explained_variance_ratio_)

# print("components", pca.components_)

pca_top_features = [df.columns[i] for i in pca.components_.argsort()[:1, :30]][0]
print("\n\ntop features", pca_top_features)

variance ratio [0.29577822 0.04869341 0.03333378 0.01675    0.01449951 0.01220008
 0.01161551 0.00785954 0.00729409 0.00640559 0.00567038 0.00490737
 0.00473793 0.00435979 0.00422605 0.00354309 0.00341826 0.00324407
 0.00309485 0.00295891 0.00272074 0.0026422  0.00247779 0.0024453
 0.00232853 0.00224948 0.00221566 0.00219201 0.00208967 0.0020746 ]


top features Index(['MIMAT0004602', 'MIMAT0027614', 'MIMAT0019710', 'MIMAT0015064',
       'MIMAT0003240', 'MIMAT0004592', 'MIMAT0019015', 'MIMAT0004508',
       'MIMAT0005866', 'MIMAT0019852', 'MIMAT0027550', 'MIMAT0027452',
       'MIMAT0027468', 'MIMAT0027409', 'MIMAT0022717', 'MIMAT0025458',
       'MIMAT0027477', 'MIMAT0019214', 'MIMAT0024610', 'MIMAT0027461',
       'MIMAT0027681', 'MIMAT0028117', 'MIMAT0026477', 'MIMAT0027500',
       'MIMAT0004951', 'MIMAT0004982', 'MIMAT0022945', 'MIMAT0027453',
       'MIMAT0027665', 'MIMAT0003887'],
      dtype='object')


In [ ]:
pca_top_features_file =  "/content/drive/MyDrive/Colab Notebooks/pca_top_features_file.csv"
df_pca_top_features = pd.DataFrame(pca_top_features)
df_pca_top_features.to_csv(pca_top_features_file, index=False)

In [ ]:

# Setup arrays to store training and test accuracies
neighbors = np.arange(1, 9)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))


In [ ]:

for i, k in enumerate(neighbors):
    # Setup a knn classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)

    # Fit the model
    knn.fit(x_train, y_train)

    # Compute accuracy on the training set
    train_accuracy[i] = knn.score(x_train, y_train)

    # Compute accuracy on the test set
    test_accuracy[i] = knn.score(x_test, y_test)
